### ideas
- We *could* do PCA to vizualize word2vec technology 

# GloVe with keras classification
##### Content:
- Import pretrained GloVe vectorspace
- Import our own data
- classify with keras FFNN(feedforward)


##### possible additional steps
- clustering
- preprossessing
- tf-idf
- experiment with different neural networks
- PCA vizualize vectorspace
- visualize end results


In [7]:
# external imports
import numpy as np
import pandas as pd
import seaborn as sb
import gensim
from gensim.scripts.glove2word2vec import glove2word2vec
import pylab as pl
import matplotlib.pyplot as plt
import csv
import scipy
import os.path
import sklearn as sk
import keras
from keras.layers import *
from keras.layers.core import *

# internal imports
import helpers as HL
import cleaning as CL
import glove_module as GV
import neural_nets as NN



# Constants
DATA_FOLDER = os.path.join("..", "..","glove_twitter_27B") #Spesialt for min location..opsops
DATA_25DIM = DATA_FOLDER + "/glove_twitter_27B_25d.txt"
DATA_50DIM = DATA_FOLDER + "/glove_twitter_27B_50d.txt"
DATA_100DIM = DATA_FOLDER + "/glove_twitter_27B_100d.txt"
DATA_200DIM = DATA_FOLDER + "/glove_twitter_27B_200d.txt"

## Import pretrained GloVe with gensim
one can use gensims word2vec functions to check similarity and other interesting functions
https://radimrehurek.com/gensim/models/word2vec.html

In [6]:
GV.method1("gensim_glove_vectors_200dim.txt", full_corpus, total_training_tweets, nr_pos_tweets, all_neural_nets=True)


Model:  basic_model
83.14% (+/- 0.79%)
Negative sentiment: 81.73%  Positive sentiment: 84.54%
Percentage of positive classifications (should be 50%ish): 51.4024898454
Time taken:  0.7351263125737508 

Model:  basic_model_adam
83.12% (+/- 0.78%)
Negative sentiment: 80.92%  Positive sentiment: 85.31%
Percentage of positive classifications (should be 50%ish): 52.1930112953
Time taken:  0.7737044175465901 

Model:  wide_model
83.39% (+/- 0.92%)
Negative sentiment: 81.51%  Positive sentiment: 85.27%
Percentage of positive classifications (should be 50%ish): 51.8824998204
Time taken:  0.9720427632331848 

Model:  deep_2_model
83.28% (+/- 1.00%)
Negative sentiment: 81.59%  Positive sentiment: 84.97%
Percentage of positive classifications (should be 50%ish): 51.6894661507
Time taken:  1.1689743677775064 

Model:  deep_HB
83.76% (+/- 1.31%)
Negative sentiment: 82.32%  Positive sentiment: 85.19%
Percentage of positive classifications (should be 50%ish): 51.4344948303
Time taken:  1.6591484308242

In [ ]:
# ONLY NEED TO THIS THE FIRST TIME ONE IMPORTS THE PRETRAINED GLOVE
# Creates a gensim_word2vec_file in the same folder
# GV.create_gensim_word2vec_file(DATA_25DIM)

In [8]:
# uses the created gensim-.txt file to create the word2vec so one can operate on it
global_vectors = GV.make_glove("gensim_glove_vectors_25dim.txt")

In [ ]:
# supposed to show queen, to demonstrate the power of word2vec, but failed #blameTwitterDataset
#global_vectors.most_similar(positive=['woman', 'king'], negative=['man'])

## Get own data ready for classification

#### Some variables

In [10]:

#FOR TRAINING_SET
corpus_filenames = ['train_pos.txt', 'train_neg.txt','test_data.txt'] 
nr_pos_tweets = 100000
nr_neg_tweets = 100000
total_training_tweets = 200000

#### Create corpus from files

In [11]:
full_corpus, corpus_file_lengths = HL.create_corpus(corpus_filenames)

print("Length full corpus", len(full_corpus))
print("File lengths:", corpus_file_lengths)

Length full corpus 210000
File lengths: [100000, 100000, 10000]


# TESTING COMPLETE FUNCTION

In [12]:
models2 = [NN.basic_model, NN.basic_model_adam, NN.wide_model, NN.deep_2_model, NN.deep_HB]

In [14]:
GV.classify_with_neural_networks(models2, global_vectors, full_corpus, total_training_tweets, nr_pos_tweets)

AttributeError: 'NoneType' object has no attribute 'syn0'

#### Tokenizing hedda-style

In [ ]:
import tolken
tokenized_full_corpus = tolken.replace_words(full_corpus)

#### N-gram

In [ ]:
ngram_corpus = HL.creating_n_grams_cropus(corpus=tokenized_full_corpus, n_gram=2) #2GramsForLife 

#### Creating labels for the training files. Used to perform validation of the predictions

In [ ]:
#Making labels
labels = np.zeros(total_training_tweets);
labels[0:nr_pos_tweets]=1;
labels[nr_pos_tweets:total_training_tweets]=0; 

print("labels shape: ", labels.shape)

#### Splitting the corpus into train and prediction - parts

We're done training the word2vec, so all "common" operations are finished

In [ ]:
# SPLITTING THE W2V model into training and predict
train_clusterised_corpus = tokenized_full_corpus[:total_training_tweets:]
predict_clusterised_corpus = tokenized_full_corpus[total_training_tweets::]

print(len(train_clusterised_corpus))
print(len(predict_clusterised_corpus))

In [ ]:
train_document_vecs = np.concatenate([GV.buildWordVector(z, global_vectors.syn0.shape[1], global_vectors) for z in train_clusterised_corpus])
train_document_vecs = sk.preprocessing.scale(train_document_vecs)

In [ ]:
print("Train w2v shape:",train_document_vecs.shape)

In [ ]:
print(labels)

## Running the neural net classifier

In [ ]:
#TING FOR Å SIKRE REPRODUSERBARHET ( ikke alt er nødv. nødvendig )

import os
import random
import time
os.environ['PYTHONHASHSEED'] = '0'
random.seed(12345) # NO IDEA WHAT THIS DOES

seed = 7
np.random.seed(seed)

#### Defining some neural net models

#### Crossvalidate this bitch

In [ ]:
def run_k_fold(models, X, Y, epochs, n_folds, seed):
    
    for neural_model in models:
        
        model_name = neural_model.__name__
        
        model = neural_model()
        
        start = time.time()

        kfold = sklearn.model_selection.StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=seed)
        cv_scores = []
        
        for train, test in kfold.split(X, Y):
            
            early_stopping = keras.callbacks.EarlyStopping(monitor='loss', patience=3)
            
            model.fit(X[train], Y[train], epochs=epochs, batch_size=1024, verbose=0, callbacks=[early_stopping])
            
            score = model.evaluate(X[test], Y[test], verbose=0)
            cv_scores.append(score)
        
        
        # To analyze if it is unbalanced classifying
        labels = Y[test]
        pred = model.predict(X[test])
        pos_right = 0
        neg_right = 0
        for i, label in enumerate(labels):
            if label == 1 and label == pred[i]:
                pos_right += 1
            elif label == 0 and label == pred[i]:
                neg_right += 1
        pos_perc = pos_right/(len(labels)*0.5)
        neg_perc = neg_right/(len(labels)*0.5)
        
        print("Model: ", model_name)
        print("%.2f%% (+/- %.2f%%)" % (np.mean(cv_scores), np.std(cv_scores)))
        print("Negative sentiment: %.2f%%  Positive sentiment: %.2f%%" % (neg_perc, pos_perc))
        print("Time taken: ", (time.time() - start) / 60, "\n")

In [ ]:
models = [ basic_model, basic_model_adam, wide_model, deep_1_model, deep_2_model, deep_HB]#, recurrent_model, convolutional_model]

run_k_fold(models, train_document_vecs, labels, epochs=10, n_folds=3, seed=7)

In [ ]:
pos_scores = []
neg_scores = []
labs = [0,1,1,1,1,0]
prd = [0.32,0.49,0.50,0.6232,0.81,0.9]
pos_right = 0
neg_right = 0
for i, lab in enumerate(labs):
    if lab == 1 and prd[i] >=0.5:
        pos_right += 1
        
    elif lab == 0 and prd[i] < 0.5:
        neg_right += 1
pos_scores.append((pos_right / (len(labs) * 0.5))*100)
neg_scores.append((neg_right / (len(labs) * 0.5))*100)
print("neggies:", np.mean(neg_scores))
print("possies:", np.mean(pos_scores))

In [ ]:
n_labs = np.array(labs)
n_prd = np.array(prd)
n_prd = np.round(prd)
print(n_prd.astype(int))
arg1 = np.argwhere(n_labs == 1)
arg0 = np.argwhere(n_labs == 0)
neg_

print(pos_corr)

In [ ]:
print("Negative sentiment: %.2f%%  Positive sentiment: %.2f%%" % labels[i==0][np.argmax(pred)]/len(labels))#, pred[0][np.argmax(pred)] * 100))

In [ ]:
models2 = [NN.basic_model, NN.basic_model_adam, NN.wide_model, NN.deep_2_model, NN.deep_HB]


In [ ]:
GV.classify_with_neural_networks(models, global_vectors, processed_corpus)